<a href="https://colab.research.google.com/github/ynaowusu/protein-folding-quantum-algorithms/blob/main/Copy_of_proteinfolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit #this is just me intalling qiskit into our notebook
!pip install matplotlib plotly #since it says we need a 3d structure to simulate the lattice and any other 3d elements
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random


In [ ]:
class ProtienFoldVector: #added a class here because the vector encdoings can be seen as an object
  def __init_(self):
    self.lattice_vector = {
        "A":{
            0:[(1,1,1)], #|00> This is for the first round of qubits reprsents x,y,z
            1:[(1,-1,-1)], #|10>
            2:[(-1,1,-1)], #|01>
            3:[(-1,-1,-1)] #|11>
        },
        "B":{
            0:[(1,1,-1)], #|00>
            1:[(1,-1,1)], #|10>
            2:[(-1,1,1)], #|01>
            3:[(-1,-1,-1)] #11>
        }
    }


        #proteins fold in 3D space and here we're working with a simplified lattice model w tetrahedral directions. There are two kinds of alternating sites in the protein chain: 'A' and 'B'.
        #every turn corresponds to a direction in 3D space either x,y or z
        #The vectors  [1, 1, 1]) are unit steps in different directions based on the protein structure model.







    pass


    #vectors go here


In [ ]:
#hamiltonian functions


# t_i and t_j are vectors
# we decode the bit string and supply indices i and j directly.
#for each pair of beads i and j, T(i,j ) returns a 1 iff the turns t1 and tj are along the same axis
#we need this for the growth constrainst function
#we need this to prevrent the growth of the chain towards unphysical geometries



def T(t_i,t_j):
  if np.array_equal(t_i,t_j) or np.array_equal(t_i, -t_j) :
    return 1
  else:
    return 0

def test_T_function():
  assert T(np.array([1,1,1]),np.array([1,1,1]))== 1
  assert T(np.array([-1,-1,-1]),np.array([-1,-1,-1]))== 1
  assert T(np.array([1,1,1]),np.array([-1,-1,-1]))== 1

  assert T(np.array([1,-1,-1]),np.array([1,1,-1]))== 0
  assert T(np.array([-1,1,-1]),np.array([-1,-1,-1]))== 0

  print("all test_t_function tests passed")
  test_T_function()

'''
because there are somethings we do not have implemented yet, growth_constraint_hamiltonian is a bit hardcoded
what growth constaint does is it eliminates sequences where the same axis
is chosen twice in a row, since this will give rise to a chain folding back onto itself.
turns is for now a placeholder but it  supposed to be a chain and so we will decode the measured qubits to then get the backbone turns
for the particular fold and then if some consecutive pairs share an axis, then the penalty which here is 30 will apply
N is the length of the peptide chain which is the number of amino-acid residues

'''
amino_acid_sequence = ([1,2,3,4,5,6])
N = random.choice(amino_acid_sequence)
def growth_constraint_Hamiltonian():
  H_gc = 0
  penalty_weight = 30
  turns = [np.array([1,1,1]),np.array([1,1,1]), np.array([1,-1,-1]),np.array([-1,-1,1]),np.array([1,1,-1]),np.array([1, -1,-1]),np.array([-1,-1,1])]
  for i in range(3, N - 1):
     same_axis = T(turns[i], turns[i + 1])
     H_gc += 30 * same_axis
  return H_gc
growth_constraint_Hamiltonian()

In [ ]:
#the methods we creae will be used to fold protein model with 6 and 8
#amino acid sequences on 3D lattice
amino_acid_sequence = ([6,7,8])
N = random.choice(amino_acid_sequence)
print(N)

NameError: name 'random' is not defined

In [ ]:
class CVARVQE:
   """
    This is a special kind of quantum optimizer that we got from the paper written by Robert
    instead of averaging all results, it focuses only on the best lowest energy outcomes which makes it better for understanding prolems like protien folding
    """
    def __init__(self, hamiltonian, alpha = 0.1):
      #self is just the initializer
      #the hamiltonain in this case calcualtes the amount of energy it takes to formulate any given protien configuration.
      #